In [22]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


In [23]:
api_wrapper_wikipedia = WikipediaAPIWrapper(top_k_results= 1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia)
print(wiki)

api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\DELL\\Documents\\GitHub\\QA-ChatBot\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)


In [24]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxive = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxive)

api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)


In [25]:
tools = [wiki, arxive]

In [26]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings= HuggingFaceEmbeddings(model_name= "all-MiniLM-L6-v2")



In [27]:
loader = WebBaseLoader("https://docs.smith.langchain.com")
docs = loader.load()
splitter =  RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap= 50)
text_split = splitter.split_documents(docs)
vectordb = FAISS.from_documents(docs, embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002387BCEEA10>, search_kwargs={})

In [28]:
# Convert this retriever in tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Searh any information about langsmith")
retriever_tool.name

'langsmith-search'

In [29]:
tools = [wiki, arxive, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\DELL\\Documents\\GitHub\\QA-ChatBot\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Searh any information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002384C6B4D60>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0

In [31]:
# Run all the tools with agents and LLM Models
import openai
from langchain_huggingface import ChatHuggingFace
from langchain_groq import ChatGroq
# Use a pipeline as a high-level helper



#huggingface-cli login
#from transformers import pipeline
#pipe = pipeline("text-generation", model="meta-llama/Llama-3.1-405B")

# Load model directly
#from transformers import AutoTokenizer, AutoModelForCausalLM

#tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-405B")
#model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-405B")
# Tools LLm--> Agent Executor
# from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
import os
groq_api_key = os.getenv("GROQ_API_KEY")
#os.environ["HUGGINGFACE_TOKEN"] = os.getenv("HUGGINGFACE_TOKEN")

# openai.api_key = os.getenv("OPENAI_API_KEY")

# llm = ChatHuggingFace(llm= "all-MiniLM-L6-v2")
llm = ChatGroq(groq_api_key= groq_api_key, model= "Llama-8b-8192")


GroqError: The api_key client option must be set either by passing api_key to the client or by setting the GROQ_API_KEY environment variable

In [32]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\DELL\Documents\GitHub\QA-ChatBot\.venv\Lib\site-packages\langsmith\client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor
agent_executor.invoke({"input": "Tell me about Langsmith"})

In [ ]:
agent_executor.invoke({"input": "What is machine learing?"})